# Supervision Hack - Zadanie Karate KIID

### Zespół MAGIW: Anna Wawrzyńczak, Wiktoria Śliwińska, Izabela Telejko, Marcel Witas, Grzegorz Zbrzeżny 

In [1]:
import pdfplumber
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import os
import math
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime
import spacy
from spacy.lang.pl.examples import sentences 
import nltk
from nltk.tokenize import word_tokenize
from collections import defaultdict

# 1) Webscrapping

In [2]:
class KIIDFound(Exception):
    pass

In [3]:
def find_kiids(pdf_links, pdf_links_titles, wzor1, wzor2):
    wynik=[]
    for i in range(len(pdf_links)):
        if pdf_links_titles[i] is not None:
            link_title = pdf_links_titles[i]
        else:
            link_title = ""
        ratio1 = fuzz.partial_ratio(pdf_links[i].lower(), wzor1)
        ratio2 = fuzz.partial_ratio(pdf_links[i].lower(), wzor2)
        ratio3 = fuzz.partial_ratio(link_title.lower(), wzor1)
        ratio4 = fuzz.partial_ratio(link_title.lower(), wzor2)

        if ratio1 > 80 or ratio3 > 80 or ratio2 > 95 or ratio4 > 95:

            wynik.append(pdf_links[i])
    return wynik
wzor1 = 'kluczowe informacje dla inwestorow'
wzor2 = 'kiid'
wzor3 = 'kluczowe-informacje-dla-inwestorow'

In [4]:
def save_pdf(url):
    r = requests.get(url, stream=True)
    domain = url.split(".")[1]
    if not os.path.exists(domain):
        os.makedirs(domain, exist_ok=True) 
    nazwa = domain +'/' + url.split("/")[-1]
    with open(nazwa, 'wb') as f:
        f.write(r.content)

In [5]:
def scrap_all_kiids(url, visited: list):
    domain = url
    wynik_final = []
    endings = ["zip", "png", "xlsx", "xls", "jpg", "docx", "csv"]
    def scrape_kiid(url, visited: list):
        if len(visited)>0 and url in visited:
            return
        visited.append(url)
        if "obsluga" in url or "pomoc" in url or "wsparcie" in url or "support" in url or "komentarze" in url or "blog" in url or "forum" in url or "ogloszenia" in url or "aktualno" in url or "aktualizacja" in url or "regulamin" in url:
            return
        print(url)
        try:
            response = requests.get(url)
        except:
            return
        try:
            soup = BeautifulSoup(response.text, "html.parser")
        except:
            return
        hrefs = soup.findAll('a', href=True)
        titles = [hrefs.string for hrefs in hrefs]
        res = {}
        for key in hrefs:
            for value in titles:
                res[key["href"]] = value
                titles.remove(value)
                break
        
        array = np.unique([a["href"] for a in hrefs])
        if len(array) == 0:
            return
        no_https = array[~np.char.startswith(array, "http")]
        pdf_links = no_https[np.char.endswith(no_https, "pdf")].tolist()
        pdf_links_titles = list(map(res.get, pdf_links))
        no_https = no_https[~np.char.endswith(no_https, "pdf")]
        inner_site_links = no_https[np.char.startswith(no_https, "/")]
        for ends in endings:
             inner_site_links = inner_site_links[~np.char.endswith(inner_site_links, ends)]
        inner_site_links = inner_site_links.tolist()
        wynik = find_kiids(pdf_links, pdf_links_titles, wzor1 = 'kluczowe informacje dla inwestorow', wzor2 = 'kiid')
        if len(wynik) > 1:
            for wyniczek in wynik:
                old_date = min([int(i) for i in re.findall("20\d{2}", wyniczek)])
                if old_date < 2022:
                    wynik.remove(wyniczek)
            print(wynik)
            wynik_final.extend([domain[:-1]+wynik for wynik in wynik])
            raise KIIDFound
        visited.append(url)
        for link in inner_site_links:
            scrape_kiid(domain[:-1]+link, visited)
    try:
        scrape_kiid(url, visited)
    except KIIDFound:
        for pdf in wynik_final:
            save_pdf(pdf)


In [6]:
domains = ['https://www.amundi.pl/',
 'https://www.baltictfi.pl/',
 'https://www.tfi.bnpparibas.pl/',
 'https://www.bpstfi.pl/',
 'https://www.caspar.com.pl/',
 'https://www.eitfi.pl/',
 'https://www.esaliens.pl/',
 'https://www.generali-investments.pl/',
 'https://www.investors.pl/',
 'https://www.ipopema.pl/',
 'https://www.millenniumtfi.pl/',
 'https://www.nn.pl/',
 'https://www.nntfi.pl/',
 'https://www.noblefunds.pl/',
 'http://www.opera.pl/',
 'https://www.pekaotfi.pl/',
 'https://www.pfrtfi.pl/',
 'https://www.pkotfi.pl/',
 'https://www.qtfi.pl/',
 'https://www.rockbridge.pl/',
 'https://www.santander.pl/tfi/',
 'https://www.skarbiec.pl/',
 'https://www.superfund.pl/',
 'https://www.franklintempleton.pl/',
 'https://www.tfiagro.pl/',
 'https://www.allianz.pl/',
 'https://www.tfienergia.pl/',
 'https://www.pzu.pl/',
 'https://www.uniqa.pl/',
 'https://www.vigcq-tfi.p/']

In [8]:
def scrapp_pages(domains):
    for domain in domains:
        visited = []
        wynik_final = []
        start = datetime.now()
        scrap_all_kiids(domain, visited)
        end = datetime.now()
        print(domain + ": " + str(end-start))

In [ ]:
scrapp_pages(domains)

https://www.eitfi.pl/
https://www.eitfi.pl/accept_cookies
https://www.eitfi.pl/akcjonariusze
https://www.eitfi.pl/bezpieczne-logowanie
https://www.eitfi.pl/dokumenty
https://www.eitfi.pl/en
https://www.eitfi.pl/en/
https://www.eitfi.pl/en/about-us
https://www.eitfi.pl/en/contact-us
https://www.eitfi.pl/en/cookies-policy
https://www.eitfi.pl/en/corporate-governance
https://www.eitfi.pl/en/dedicated-funds
https://www.eitfi.pl/en/disclaimer
https://www.eitfi.pl/en/managed-funds
https://www.eitfi.pl/en/performance
https://www.eitfi.pl/en/user/first-login
https://www.eitfi.pl/en/user/login
https://www.eitfi.pl/en/user/password-reset-email
https://www.eitfi.pl/informacja-esg
https://www.eitfi.pl/fundusze-prywatne
https://www.eitfi.pl/fundusze-zarzadzane
https://www.eitfi.pl/kariera
https://www.eitfi.pl/kontakt
https://www.eitfi.pl/lad-korporacyjny
https://www.eitfi.pl/nota-prawna
https://www.eitfi.pl/notowania
https://www.eitfi.pl/o-firmie
https://www.eitfi.pl/partnerzy
https://www.eitfi.pl/

# Lista plików z KIIDami 

## Tworzenie tabeli - MAGIW_KIID_META.csv

In [9]:
def create_kiid_list():
    kiids_list = []
    for i in os.walk(os.getcwd()):
        if("ipynb" in i[0] or i[0] == os.getcwd()):
            continue
        for file in os.listdir(i[0]):
            if file.split(".")[-1] == "pdf":
                kiids_list.append(file)
    df = pd.DataFrame({"ID_KIID": [i for i in range(1, len(kiids_list) + 1)], "ID_ZESPOLU": [6 for i in range(1, len(kiids_list) + 1)], "NAZWA_PLIKU": kiids_list})
    return df

In [10]:
MAGIW_KIID_META = create_kiid_list()
if not os.path.exists('pliki'):
    os.mkdir('pliki')
MAGIW_KIID_META.to_csv('pliki/MAGIV_KIID_META.csv')

# 2) Podstawowe informacje o KIIDach

## Tworzenie tabeli - MAGIW_KIID_DANE.csv

In [20]:
kiids = []
names = []
for i in os.walk(os.getcwd()):
    if("ipynb" in i[0] or i[0] == os.getcwd()):
        continue
    for file in os.listdir(i[0]):
        if file.split(".")[-1] == "pdf":
            with pdfplumber.open(i[0] + "\\" + file) as pdf:
                page1 = pdf.pages[0]
                page2 = pdf.pages[1]
                tekst = (page1.extract_text() + page2.extract_text())
                kiids.append(" ".join(tekst.replace("\n[0-9]", " ").replace("\n", " ").replace(":", "").replace(";", "").strip().split()))
                names.append(file)

In [21]:
def create_data_table(kiids, MAGIW_KIID_META):
    MAGIW_KIID_DANE = None
    for i, tekst in enumerate(kiids):
        #id kiid
        id_kiid = MAGIW_KIID_META.loc[MAGIW_KIID_META["NAZWA_PLIKU"] == names[i]][["ID_KIID"]].iloc[0, 0]

        #subfundusz
        fundusz = None
        try:
            info = re.split(r"inwestycyjną.|Cele i polityka inwestycyjna", tekst)[1].replace(",", "").split(" ")[1:]
            info = [inf for inf in info if inf not in ['Subfundusz', 'Kategoria', 'Jednostka']]
            fundusz = ''
            for word in info:
                if len(word) > 2:
                    if not "." in word and (all(c.isalpha() for c in word) and word[0].isupper()) or (not any(c.isalpha() for c in word) ):
                        fundusz = fundusz + " " + word
                    elif len(fundusz) > 0:
                        break
        except:
            pass
        #MAGIW_KIID_DANE.loc[i, "NAZWA_SUBFUNDUSZU"] = fundusz

        # isin
        isin = None
        try:
            isin_list = tekst.split("ISIN",  maxsplit=1)[1].split(" ")
            for word in isin_list:
                if len(word) == 12 and any([char.isdigit() for char in word]):
                    isin = word
                    break
            #MAGIW_KIID_DANE.loc[i, "ISIN"] = isin 
        except:
            pass

        #identyfikator krajowy
        idkr = None
        try:
            idkr_list = tekst.split("identyfikator",  maxsplit=1)[1].split(" ")
            for word in idkr_list:
                if len(word) == 11 and any([char.isdigit() for char in word]):
                    idkr = word
                    break
            #MAGIW_KIID_DANE.loc[i, "IDENTYFIKATOR_KRAJOWY"] = idkr
        except:
            pass

        #numer RFi
        rfi = None
        try:
            rfi_list = tekst.split("RFi",  maxsplit=1)[1].split(" ")
            for word in rfi_list:
                if len(word) in [3, 4] and all([char.isdigit() for char in word]):
                    rfi = word
                    break
            #MAGIW_KIID_DANE.loc[i, "NUMER_RFI"] = rfi
        except:
            pass

        #podmiot zarządzający
        pod = None
        try:
            index_pod2 = tekst.find("S.A.") 
            if index_pod2 != -1:
                pod = tekst[:index_pod2+4]
                try:
                    pod = re.split(r"arządzany przez spółkę|arządzany przez|półka zarządzająca|arządzany jest przez", pod)[-1]
                    #MAGIW_KIID_DANE.loc[i, "PODMIOT_ZARZADZAJACY"] = pod
                except:
                    pass
        except:
            pass
        try:
            index_pod2 = tekst.find(" SA ") 
            if index_pod2 != -1:
                pod = tekst[:index_pod2+3]
                try:
                    pod = re.split(r"arządzany przez spółkę|arządzany przez|półka zarządzająca|arządzany jest przez", pod)[-1]
                    #MAGIW_KIID_DANE.loc[i, "PODMIOT_ZARZADZAJACY"] = pod
                except:
                    pass
        except:
            pass

        #cel inwestycyjny i polityka inwestycyjna
        cel = None
        try:
            index_cel = tekst.find("Cel inwestycyjny") 
            if index_cel != -1:
                cel = tekst[index_cel:]
                index_cel2 = [i for i,c in enumerate(cel) if c=='.'][1]
                cel = cel[:index_cel2+1]
                #MAGIW_KIID_DANE.loc[i, "CEL_INWESTYCYJNY"] = cel
        except:
            pass
        try:
            index_cel = tekst.find("Celem inwestycyjnym")
            if index_cel != -1:
                cel = tekst[index_cel:]
                index_cel2 = [i for i,c in enumerate(cel) if c=='.'][1]
                cel = cel[:index_cel2+1]
                #MAGIW_KIID_DANE.loc[i, "CEL_INWESTYCYJNY"] = cel
        except: 
            pass
        pol = None
        try:
            index_pol = tekst.find("Polityka inwestycyjna")
            if index_pol != -1:
                pol = tekst[index_pol:]
                index_pol2 = [i for i,c in enumerate(cel) if c=='.'][1]
                pol = pol[:index_pol2+1]
                #MAGIW_KIID_DANE.loc[i, "POLITYKA_INWESTYCYJNA"] = pol
        except:
            pass

        #kategoria 
        cat = []
        try:
            name = names[i]
            index_list = [i for i,c in enumerate(name) if c=='-']
            for ind in index_list:
                if name[ind-1].isalpha() and name[ind-1].isupper():
                    cat.append(name[ind-1])
                if name[ind+1].isalpha() and name[ind+1].isupper() and not (name[ind+2].isalpha() and name[ind+2].islower()):
                    cat.append(name[ind+1])
            cat = np.unique(cat)
        except:
            pass
        if len(cat) == 0:
            try:
                category = tekst.split("ategori").split(" ").replace(",", "")
                print(category)
            except:
                pass

        # FIO/SFIO
        fio = None
        try:
            sf_tekst = tekst.lower()
            if "specjalistyczny fundusz inwestycyjny otwarty" in sf_tekst:
                fio = "SFIO"
            elif "fundusz inwestycyjny otwarty" in sf_tekst:
                fio = "FIO"
            #MAGIW_KIID_DANE.loc[i, "TYP_FUNDUSZU"] = wartosc
        except: 
            pass

        # WALUTA_KAPITALU_ZAKLADOWEGO_TOWARZYSTWA
        waluta = None
        try:
            wkzt_tekst = tekst.lower().split("Wyniki osiągnięte w przeszłości".lower())[1].split("Informacje praktyczne".lower())[0]
            if ("pln" in wktz_tekst or "złotówek" in wkzt_tekst or "zł" in wktz_tekst or "złotych" in wktz_tekst):
                waluta = "PLN"
            elif ("euro" in wkzt_tekst):
                waluta = "EURO"
            elif ("dol" in wkzt_tekst or "$" in wkzt_tekst or "dolarów" in wkzt_tekst or "dolar" in wkzt_tekst):
                waluta = "DOLAR"
            #MAGIW_KIID_DANE.loc[i, "WALUTA_KAPITALU_ZAKLADOWEGO_TOWARZYSTWA"] = waluta
        except:
            pass

        # NIP
        nip = None
        try:
            nip_tekst = tekst.lower().split("informacje praktyczne")[1]
            nipy = re.findall("\d-\d+-\d+-\d+",nip_tekst)
            if len(nipy) > 0:
                nip = nipy[0]
            #MAGIW_KIID_DANE.loc[i, "NIP"] = nip
        except:
            pass

        # KRS
        krs = None
        try:
            krs_tekst = tekst.lower().split("informacje praktyczne")[1]
            krsy = re.findall("^\d{10}$")
            if len(krsy) > 0:
                krs = krsy[0]
            #MAGIW_KIID_DANE.loc[i, "KRS"] = krs
        except:
            pass

        # DATA_AKTUALIZACJI_KIID
        date = None
        try:
            miesiace = {'sty': 1,
                          'lut': 2,
                          'mar': 3,
                          'kwi': 4,
                          'maj': 5,
                          'cze': 6,
                          'lip': 7,
                          'sie': 8,
                          'wrz': 9,
                          'paź': 10,
                          'lis': 11,
                          'gru': 12}
            ind = tekst.find('Niniejsze Kluczowe informacje dla inwestorów są aktualne na dzień')
            i1 = ind + len(s)
            i2 = i1 + tekst[i1:].find('r.')
            date = tekst[i1:i2].split()
            miesiac = date[1][:3]
            m = miesiace.get(miesiac)
            date_str = str(m)+'-'+date[0]+'-'+date[2]
            date = datetime.strptime(date_str, '%m-%d-%Y')

            #MAGIW_KIID_DANE.loc[i, "DATA_AKTUALIZACJI_KIID"] = date
        except:
            pass

        # DEPOZYTARIUSZ
        depoz = None
        try:
            s1 = re.split(r"wyniki osiągnięte w przeszłości", tekst.lower())[1]#[1].replace(",", "").split(" ")[1:]
            s2 = re.split(r"informacje praktyczne", s1.lower())[1]#[1].replace(",", "").split(" ")[1
            slowa = s2.split()
            wynik =[]
            for i in range(len(slowa)):
                if fuzz.ratio(slowa[i], 'depozytariusz') > 90:
                    wynik = wynik + slowa[i:i+8]

            s = " ".join(wynik[1:])
            ind = s.find("a.")
            depoz = s[:ind+2]
            #MAGIW_KIID_DANE.loc[i, "DEPOZYTARIUSZ"] = depoz
        except:
            pass

        # PROFIL_RYZYKA_I_ZYSKU
        t2 = None
        try:
            t1 = tekst.lower().split('profil ryzyka i zysku')[1]
            t2 = t1.lower().split('opłaty')[0] 
            #MAGIW_KIID_DANE['PROFIL_RYZYKA_I_ZYSKU'] = t2
        except:
            pass 

        #BENCHMARK  
        bench = None
        try:
            ind = tekst.find('benchmark')
            st = tekst[ind:ind+500]
            pro = re.findall("\d{2}%",st)
            bench = pro[0] + ' WIG20 + ' + pro[1] + ' mWIG40 + ' + pro[2] + ' WIBID ON'
            #MAGIW_KIID_DANE['BENCHMARK'] = bench
        except:
            pass
        if len(cat) == 0:
            cat = [None]
        for c in cat:

            new_df = {
                      "ID_KIID": id_kiid,
                      "ID_ZESPOLU": 6,
                      "NAZWA_SUBFUNDUSZU": fundusz,
                      "NAZWA_FUNDUSZU": None,
                      "ISIN": isin,
                      "IDENTYFIKATOR_KRAJOWY": idkr,
                      "NUMER_RFI": rfi, 
                      "PODMIOT_ZARZADZAJACY": pod, 
                      "DATA_AKTUALIZACJI_KIID": date,
                      "KATEGORIE_JEDNOSTEK_UCZESTNICTWA": c, 
                      "CEL_INWESTYCYJNY": cel, 
                      "POLITYKA_INWESTYCYJNA": pol ,
                      "MINIMALNY_POZIOM_INWESTYCJI_UDZIALOWE": None,
                      "MAKSYMALNY_POZIOM_INWESTYCJI_UDZIALOWE": None,
                      "MINIMALNY_POZIOM_INWESTYCJI_DLUZNE": None,
                      "MAKSYMALNY_POZIOM_INWESTYCJI_DLUZNE": None,
                      "MINIMALNY_POZIOM_INWESTYCJI_TYTULY_UCZESTNICTWA": None,
                      "MAKSYMALNY_POZIOM_INWESTYCJI_TYTULY_UCZESTNICTWA": None,
                      "CZESTOTLIWOSC_ZBYWANIA_I_ODKUPOWANIA_JEDNOSTEK_UCZESTNICTWA": None,
                      "CZY_FUNDUSZ_WYPLACA_DYWIDENDE": None,
                      "BENCHMARK": bench,
                      "ZALECANY_OKRES_INWESTYCJI": None,
                      "PROFIL_RYZYKA_I_ZYSKU": t2,
                      "SRRI": None,
                      "OPLATY": None,
                      "OPLATA_ZA_NABYCIE": None,
                      "OPLATA_ZA_ODKUPIENIE": None,
                      "OPLATY_BIEZACE": None,
                      "OPLATY_ZA_WYNIKI": None,
                      "WYNIKI_OSIAGNIETE_W_PRZESZLOSCI": None,
                      "STOPA_ZWROTU_2012": None,
                      "STOPA_ZWROTU_2012_BENCHMARK": None,
                      "STOPA_ZWROTU_2013": None,
                      "STOPA_ZWROTU_2013_BENCHMARK": None,
                      "STOPA_ZWROTU_2014": None,
                      "STOPA_ZWROTU_2014_BENCHMARK": None,
                      "STOPA_ZWROTU_2015": None,
                      "STOPA_ZWROTU_2015_BENCHMARK": None,
                      "STOPA_ZWROTU_2016": None,
                      "STOPA_ZWROTU_2016_BENCHMARK": None,
                      "STOPA_ZWROTU_2017": None,
                      "STOPA_ZWROTU_2017_BENCHMARK": None,
                      "STOPA_ZWROTU_2018": None,
                      "STOPA_ZWROTU_2018_BENCHMARK": None,
                      "STOPA_ZWROTU_2019": None,
                      "STOPA_ZWROTU_2019_BENCHMARK": None,
                      "STOPA_ZWROTU_2020": None,
                      "STOPA_ZWROTU_2020_BENCHMARK": None,
                      "STOPA_ZWROTU_2021": None,
                      "STOPA_ZWROTU_2021_BENCHMARK": None,
                      "DATA_PIERWSZEJ_WYCENY": None,
                      "DEPOZYTARIUSZ": depoz,
                      "KRS_TOWARZYSTWA": krs,
                      "NIP_TOWARZYSTWA": nip,
                      "SIEDZIBA_TOWARZYSTWA": None,
                      "KAPITAL_ZAKLADOWY_TOWARZYSTWA": None,
                      "WALUTA_KAPITALU_ZAKLADOWEGO_TOWARZYSTWA": waluta,
                      "CZY_ESG": None,
                      "TYP_FUNDUSZU": fio
            }
            if MAGIW_KIID_DANE is None:
                MAGIW_KIID_DANE = pd.DataFrame(new_df, index=[0])
            else:
                MAGIW_KIID_DANE = MAGIW_KIID_DANE.append(new_df, ignore_index=True)
    return MAGIW_KIID_DANE

In [22]:
MAGIW_KIID_DANE = create_data_table(kiids, MAGIW_KIID_META)
if not os.path.exists('pliki'):
    os.mkdir('pliki')
MAGIW_KIID_DANE.to_csv('pliki/MAGIW_KIID_DANE.csv')

# 3) Tablica słów występujących we wszystkich dokumentach KIID 

## Tworzenie tabeli MAGIW_KIID_BAGOFWORDS_S.csv

In [19]:
nlp = spacy.load("pl_core_news_sm")
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lukgr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
def bag_of_words(texts):
    vocab = []
    cały_tekst = ""
    for i in range(0, len(texts)):
        cały_tekst += texts[i] 
    count_dict = defaultdict(int)
     x = word_tokenize(cały_tekst)
    for i in range(0, len(x)):
        x[i] = x[i].lower()
    punctuations="?:!.,; ()%\/?"
    for word in x:
        if word in punctuations:
            x.remove(word)
    vocab = []
    for word in x:
        if word not in vocab:
            vocab.append(word)
    for word in x:
        if len(x) == 1:
            vocab.remove(x)
    index_word = {}
    i = 0
    for word in vocab:
        index_word[word] = i 
        i += 1
    vec = np.zeros(len(vocab))
    for item in x:
        count_dict[item] += 1
    for key,item in count_dict.items():
        vec[index_word[key]] = item
    bow = vec.tolist()
    for i in range(0, len(bow)):
        bow[i] = int(bow[i])
    df = pd.concat([pd.DataFrame({"ID_KIID": [i for i in range(1, len(vocab) + 1)],
                              "ID_ZESPOLU": [6 for i in range(1, len(vocab) + 1)]}),
                        pd.DataFrame(columns = ["SLOWO",
                                          "LICZBA WYSTAPIEN"
                            
    ])], axis=1)
    for i in range(0, len(vocab)):
        df.loc[i, "SLOWO"] = vocab[i]
        df.loc[i, "LICZBA WYSTAPIEN"] = bow[i]
    return df

In [ ]:
MAGIW_KIID_BAGOFWORDS_S = bag_of_words(kiids)
if not os.path.exists('pliki'):
    os.mkdir('pliki')
MAGIW_KIID_BAGOFWORDS_S.to_csv('pliki/MAGIW_KIID_BAGOFWORDS_S.csv')

## Tworzenietabeli MAGIW_KIID_BAGOFWORDS_N.csv

In [ ]:
nlp = spacy.load("pl_core_news_sm")

In [ ]:
def bag_of_words2(texts):
    cały_tekst = ""
    for i in range(0, len(texts)):
        cały_tekst += texts[i] 
    doc = nlp(cały_tekst, disable=["parser", "ner", "tagger"])
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    punctuations="?:!.,; ()%\/?"
    for word in lemma_list:
        if word in punctuations:
            lemma_list.remove(word)
    for word in lemma_list:
        if len(word) == 1:
            lemma_list.remove(word)
    for i in range(0, len(lemma_list)):
        lemma_list[i] = lemma_list[i].lower()
    vocab2 = []
    for word in lemma_list:
        if word not in vocab2:
            vocab2.append(word)
    index_word2= {}
    i = 0
    for word in vocab2:
        index_word2[word] = i 
        i += 1 
    count_dict = defaultdict(int)
    vec = np.zeros(len(vocab2))
    for item in lemma_list:
        count_dict[item] += 1
    for key,item in count_dict.items():
        vec[index_word2[key]] = item
    nbow = vec.tolist() 
    for i in range(0, len(nbow)):
        nbow[i] = int(nbow[i])
    for i in range(0, len(lemma_list)):
        lemma_list[i] = str(lemma_list[i])
        df = pd.concat([pd.DataFrame({"ID_KIID": [i for i in range(1, len(vocab2) + 1)],
                                  "ID_ZESPOLU": [6 for i in range(1, len(vocab2) + 1)]}),
                            pd.DataFrame(columns = ["SLOWO",
                                              "LICZBA WYSTAPIEN"

    ])], axis=1)
    for i in range(0, len(vocab2)):
        df.loc[i, "SLOWO"] = vocab2[i]
        df.loc[i, "LICZBA WYSTAPIEN"] = nbow[i]
    return df

In [ ]:
MAGIW_KIID_BAGOFWORDS_N = bag_of_words2(kiids)
if not os.path.exists('pliki'):
    os.mkdir('pliki')
MAGIW_KIID_BAGOFWORDS_N.to_csv('pliki/MAGIW_KIID_BAGOFWORDS_N.csv')

# 4) Weryfikacja obecości podstawowych wyrażeń wymaganych prawem w dokumencie KIID

## Tworzenie tabeli MAGIW_KIID_WYRAZENIA.csv

In [33]:
def kiid_dict():
    kiid_tekst = {}
    for i in os.walk(os.getcwd()):
        if("ipynb" in i[0] or i[0] == os.getcwd()):
            continue
        for file in os.listdir(i[0]):
            if file.split(".")[-1] == "pdf":
                try:
                    with pdfplumber.open(i[0] + '\\' + file) as pdf:
                        page1 = pdf.pages[0]
                        page2 = pdf.pages[1]
                        tekst = (page1.extract_text() + page2.extract_text())
                        kiid_tekst[file] = tekst.replace("\n[0-9]", " ").replace("\n", " ")
                except:
                    pass
    return kiid_tekst            

In [34]:
kiid_tekst = kiid_dict()
frazy_oblig = ["Kluczowe informacje dla inwestorów", '''Niniejszy dokument zawiera kluczowe informacje dla inwestorów dotyczące tego funduszu. Nie są to materiały marketingowe. Dostarczenie tych informacji jest wymogiem prawnym mającym na celu ułatwienie zrozumienia charakteru i ryzyka związanego z inwestowaniem w ten fundusz. Przeczytanie niniejszego dokumentu jest zalecane inwestorowi, aby mógł on podjąć świadomą decyzję inwestycyjną.''', 'Niniejsze kluczowe informacje dla inwestorów są aktualne na dzień', 'Fundusz otrzymał zezwolenie na prowadzenie działalności w', 'Zalecenie: niniejszy fundusz może nie być odpowiedni dla inwestorów, którzy planują wycofać swoje środki w ciągu', '''może zostać pociągnięta do odpowiedzialności za każde oświadczenie zawarte w niniejszym dokumencie, które wprowadza w błąd, jest niezgodne ze stanem faktycznym lub niespójne z odpowiednimi częściami prospektu emisyjnego UCITS.''', 'Opłaty jednorazowe pobierane przed lub po dokonaniu inwestycji', 'Opłata za subskrypcję', 'Opłata za umorzenie', 'Opłaty pobierane z funduszu w ciągu roku', 'Opłaty bieżące', 'Opłaty pobierane z funduszu w określonych warunkach szczególnych', 'Opłata za wyniki', 'Cele i polityka inwestycyjna', 'Profil ryzyka i zysku', 'Opłaty', 'Wyniki osiągnięte w przeszłości', ' Informacje praktyczne']

In [36]:
def szukaj_oblig(frazy_oblig, kiid_tekst):
    df_main = pd.DataFrame()
    for kiid in kiid_tekst.keys():
        tekst_split = kiid_tekst[kiid].replace("\xa0", " ").lower().split(" ")
        frazy_matchujace = []
        for fraza in frazy_oblig:
            czy_match = 0
            n = len(fraza.split(" "))
            i = 0
            while i+n <= len(tekst_split):
                zdanie_tekstu = " ".join(tekst_split[i:i+n])
                ratio = fuzz.partial_ratio(fraza.lower(), zdanie_tekstu)
                if ratio >= 95:
                    czy_match = 1
                i += 1
            frazy_matchujace.append(czy_match)
        df_main = pd.concat([df_main, pd.DataFrame({"KIID" : [kiid for i in range(len(frazy_oblig))], "Frazy obligatoryjne" : frazy_oblig, "Czy występuje" : frazy_matchujace})])
    return df_main

In [37]:
MAGIW_KIID_WYRAZENIA = szukaj_oblig(frazy_oblig, kiid_tekst)
if not os.path.exists('pliki'):
    os.mkdir('pliki')
MAGIW_KIID_WYRAZENIA.to_csv('pliki/MAGIW_KIID_WYRAZENIA.csv')
